# Labsheet 2: Motors

This worksheet will cover getting your motors to move on your 3Pi+ robot.  In previous labsheets you will have:
 - Seen the standard Arduino sketch, which has the structure of setup() and loop() routines.
 - Experimented with Arduino Example sketches
 - Uploaded sketches to your robot.
 - Experimented with Serial commands to view debug output from your code.
 
In this labsheet we are going to:

- Get the 3Pi+ Motors moving, and experiment with doing specified movements.


**Note:** The methods we develop in this labsheet are quite basic, and we will implement better methods in future labsheets.  However, this labsheet is important to introduce you to the basic concepts.  For now, do not worry about "perfect" performance - instead, try to understand and quantify the performance you see. 




<hr><br><br><br><br>

# 3Pi+ Motor Drivers

The 3Pi+ robot is controlled by an ATMEL ATMEGA32u4 Microcontroller (<a href="https://www.microchip.com/en-us/product/ATmega32U4">produce page</a>, <a href="https://ww1.microchip.com/downloads/en/DeviceDoc/Atmel-7766-8-bit-AVR-ATmega16U4-32U4_Datasheet.pdf">datasheet</a>).  We can think of a microcontroller as a computer which interacts with the external world (any connected peripherals) through `general purpose input/output` pins (GPIO pins) - the small metal tabs or legs around the actual microchip.  We can write a program (software) to tell the microcontroller to activate a pin `HIGH`, which provides a small amount of current (typically, 25 milliamps).  However, the voltage and current provided by the microcontroller is not enough to drive (activate) the motors of the 3Pi+.  

We can look at the schematic of the 3Pi+ (<a href="https://www.pololu.com/file/0J1790/3pi-plus-32u4-control-board-schematic.pdf">aviailable here</a>)to investigate how the motors can be driven.

<p align="center">
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_schematic_drv8838.png?raw=true">
</p>

In the above circuit schematic extract (found on page 2 of the linked document above), we can see the circuit symbol for the left and right motors (the blue circle on top of a blue box).  These are then attached to the integrated circuit (a microchip, the yellow box) labelled `DRV8838`, one for the left motor and one for the right motor.  

Each motor is electronically driven by a DRV8838, a low-voltage H-Bridge driver (a datasheet is available <a href="https://www.ti.com/lit/ds/symlink/drv8838.pdf?ts=1626947113238&ref_url=https%253A%252F%252Fwww.ti.com%252Fproduct%252FDRV8838">here</a>).  A <a href="https://en.wikipedia.org/wiki/H-bridge">H-Bridge</a> is a type of circuit which allows DC motors to be driven either forwards or backwards without needing to physically change the polarity of the connections to the DC motor.  

The fine black lines extending out of the yellow block indicate necessary connections to the DRV8838:  
- The DRV8838 is able to source (provide) more voltage and current through the connection labelled `VM` ("voltage-motors"), which is an alternative power supply.  This provides an independent power supply for the motors.
- The connection labelled `5V` provides a stable voltage supply for the digital logic of the DRV8838 circuit.  
- The DRV8838 has to be connected to ground (0v) to complete the electronic circuit, marked by the symbol which looks like a triangle of horizontal lines.  
- The labels `OUT1` and `OUT2` are attached to the DC motor.  By making one output `HIGH` and other `LOW`, the motor is driven forwards or backwards.
- We can observe that both the DRV8838 have connections labelled `PH` and `EN`:
  - Note: `PH` and `EN` appear to have an arrow indicated into them, but the lines are not terminated against a symbol.  This means they connect to another part of the circuit, drawn elsewhere.  
  - The red labels `D16`, `D10`, `D15`, `D9` can be found elsewhere in the document, which is their corresponding connection.  These are connections to our 32u4 microcontroller.
  - We can confirm these connections against the <a href="https://www.pololu.com/docs/0J83/5.9">pin assignment section of the documentation</a> (look for functions relating to motor).

The 3PI+ DRV8838 therefore have just two input signals each for their operation:

- The **`PH`** pin of the DRV8838 sets the **`direction`** of rotation of the motor of the 3Pi+.  In the pin assigment documentation, this is referred to as "left/right motor direction".

- The **`EN`** pin of the DRV8838 `enables` the motors.  When `EN` is set `LOW`, the motors are `not active`.  When `EN` is set `HIGH`, the motors are driven from the power supply `VM`.  In order to achieve different rotation speeds, `EN` is switched on-and-off very quickly.  In the pin assigment documentation, this is referred to as "left/right motor PWM".

`PWM` stands for Pulse Width Modulation.  Pulse Width Modulation means turning a digital signal either `HIGH` or `LOW` (`ON` or `OFF`) within a fixed period of time, and adjusting the timing interval of the signal.  This is often referred to as a `mark-to-space ratio`, because of the way the output signal appears, which forms the `duty-cycle` in %:

<p align="center">
<img width="50%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/PWM.png?raw=true">
</p>

In the above, we can see that at a 50% duty cycle, the signal has clear and equal on-periods and off-periods, or marks and spaces respectively.  When this PWM signal is measured by the microcontroller, it is effectively averaged over time.  Therefore, a PWM with a 50% duty cycle will be measured as 2.5volts. When we instruct a microcontroller to generate PWM, we are crudely synthesising an analog signal (continuous range) from a digital signal (binary range), which is often referred to as `DAC` - digital-to-analog conversion.  The DRV8838 receives the PWM output from the 32u4 microcontroller and will effect different motor power levels.  

The DRV8838 makes operating the motors very easy, we simply set a **`PWM`** level (power), and a **`direction`**.  The 3Pi+ is a **`differential drive`** robot from Pololu. This means it has two wheels as **`effectors`**, placed symmetrically either side of the longitude of the robot, to manoeuvre the robot.  Each wheel is driven by a motor each, the **`actuators`**.  
- To make the 3Pi+ move forwards or backwards the motors can be activated with the same **`PWM`** levels, set in the same **`direction`**.  
- The 3Pi+ can be made to move on an **`arc`** by activating the motors at different **`PWM`** levels, in the same **`direction`**. 
- The 3Pi+ can be made to rotate on the spot by activating the motors at different **`PWM`** levels, in different **`directions`**.

Conveniently, most microcontrollers contain dedicated hardware to generate PWM signals - meaning it is not necessary to write our own software routine to do so.  Within the Arduino programming environment, a function called **`analogWrite()`** performs this operation (<a href="https://www.arduino.cc/reference/en/language/functions/analog-io/analogwrite/">Arduino Reference</a>).  Writing a value of 255 creates a constant high signal (5v), whilst writing a 0 creates a constant low signal (0v).  Values between 0 and 255 generate square waveforms with a ratio of high (5v) and low (0v).  

It is important to note that when we set a `PWM` level as input to the DRV8838, we are setting a desired output power level but **not** speed.  **Speed** (or velocity) is a measurement, and currently we do not know the real rotation of the wheel during a period of time.  Imagine, we could set an output PWM duty cycle of 100%, but the DC motor could be broken - a measured speed of 0.



## Exercise 1: Basic Motor Operation

1. Follow the below code template, filling in the important parameters and operations:
 - you can find the appropriate pins in the <a href="https://www.pololu.com/docs/0J83/5.9">pin assignment section of the 3Pi+ documentation</a>.  
 - you will haved used the Arduino built-in functions to configure pins in Labsheet 1.

2. Check that you have assigned the left and right pins appropriately.  Does activating just the left motor in code activate the left motor in reality?

3. Check whether a `HIGH` value for the `direction` pin produces forward or backward motion of your robot.
  - It is recommended you then create and use `#define FWD LOW` (or HIGH), and `#define REV ...` definitions in your code, to make it more readable.  


```c

// Replace the ? with correct pin numbers
// https://www.pololu.com/docs/0J83/5.9
#define L_PWM_PIN ?
#define L_DIR_PIN ?
#define R_PWM_PIN ?
#define R_DIR_PIN ?

// Runs once.
void setup() {

  // Set all the motor pins as outputs.
  // ...

  // Set initial direction (HIGH/LOW)
  // for the direction pins.
  // ...

  // Set initial values for the PWM
  // Pins.
  // ...


  // Start serial, send debug text.
  Serial.begin(9600);
  delay(1000);
  Serial.println("***RESET***");

}

// Repeats.
void loop() {

  // A slow (low) level of activation to test
  // the motor operation.
  analogWrite( L_PWM_PIN, 20 );
  analogWrite( R_PWM_PIN, 20 );

  // An empty loop can block further uploads.
  // A small delay to prevent this for now.
  delay(5);
}
```



 
## Exercise 2: Getting Familiar

Although we are writing software, it is important that you learn from experience how your robot physically behaves under certain conditions.  Take some time to experiment with your code and figure out the limitations and surprises:

1. Deadband: The forward and reverse PWM values and that cannot produce movement are called a **deadband**, and can be a difficult problem to deal with when controlling motors.  What is the range of PWM values which constitute the deadband for your robot?
  - **Experimental Observations:** the following observations will be _unique to your 3Pi+_ and will be useful for future labsheets:
    - Experiment to find the **minimum** motor PWM required for your Romi to move fowards, note this down.
    - Experiment to find the **minimum** motor PWM required for your Romi to move backwards, note this down.
    - Are the above measurements the same values for forwards and backwards?
    - Are the above measurements the same for left and right motors?
    - What factors (robot or environment) may influence your above observations?

2. Drift: Because of varying tolerances in manufacturing, it is unlikely that you 3Pi+ motors will perform in an identical way.  We can evaluate this by observing the travel of the robot against a straight line.  Use the black vinyl tape to mark down a straight line on your work surface (or draw a line). Adjust your Arduino program in the following ways:
  - Set the same direction for both motors and set the same PWM value for left and right motors.
  - Set your program so that your robot will drive forwards for a fixed period of time, enough to travel the length of your line.  You can use `delay()` for this.  Your robot should then deactivate the motors indefinitely (until power-cycled or reset is pressed).  
  - Use tape or draw some marks on the work surface so that you can start your robot in the same location and rotation precisely.
  - **Experimental Observations:**
    - Activate your robot and allow it to complete the forward travel.
    - Decide how to mark the end position of the robot.
    - Measure the angle of deviation of travel from the straight line origin.
    - Measure the the distance between your robot and the straight line.
    - Repeat this process 10 times.  


2. Linearity: You may have noticed in Exercise 1 that we can hear an audible difference as the motors are set to increasing power.  Does speed increase linearly for the values set in `analogWrite()`?

- Experiment to find the motor power after which there is no discernable difference in high speed.  How might you work this out experimentally?
- Is it more important to conduct these experiments with your robot lifted from the surface (wheels rotating in free air) or on the surface?
- See if you can build up a set of data to actually measure the speed of the robot.  You might need a long marker of distance on your work surface so that you can measure a time interval accurately.
    - speed = distance / time
    - Does the power sent to your motors have a linear relationship to the speed produced?




# Exercise 3: Confident Operation

A robotic system can quickly become very complicated, and it can be hard to tell where an error is coming from.  Therefore, we should prioritise being **confident** that our code works as expected.  If we can catch all possible error conditions in software, we will have an easier time figuring out why something isn't working.

When we work on robotic systems we also have to test our assumptions about how code is functioning.  Even though code may compile with no errors, the performance (or utility) of the code is dependent on the physical operation of the robot.  

## Task

- Change the above code so that `l_power` and `r_power` are of type **float**.  Note down the following observations:
    - What happens if you send negative numbers to _analogWrite()_?
    - What happens if a value larger than 255 is sent to _analogWrite()_?
    - What happens if fractional numbers are sent to _analogWrite()_?
    - **There are no 'correct' answers to these questions.**  
        - Conduct experiments to increase your understanding of the full behaviour of <a href="https://www.arduino.cc/reference/en/language/functions/analog-io/analogwrite/">analogWrite()</a>. 
        - When we write software for robotic systems, the only way to fully understand the performance is to experiment and test your system.
        

# Exercise 4: Clean, Efficient, Readable Code

A lot of time is spent hunting down errors and bugs.  Writing clean, effcient and readable code will save you time.  

Above, we asked what happens if a negative value is sent to `analogWrite()`.  Intuitively, if we:
- Send a positive power value, we could make the motor rotate forwards.
- Send a negative power value, we could make the motor rotate backwards.

To do this, we need to use an `if statement` to check the sign of the power value, and then set the `direction pin` appropriately. 

## Task

- Start by writing some code to perform the above operation for one motor.  You can write and test this straight into `loop()`.
- Write a function for each motor that will take in a power as a signed value (positive and negative), and use the sign to set the direction for the motor.
    - setLeftMotorPower( power )
    - setRightMotorPower( power )
- From your observations in the previous exercise, write code into your function to mitigate all error conditions.

    
# Exercise 5: Exploring Limitations (optional)

**For these exercises, do not attempt to achieve perfect peformance.  Instead, explore the performance you observe until you are happy that you understand the causes sufficiently.   You should also consider how we can measure and evaluate performance.   We will implement better control techniques in a later labsheet.**
- Extend the above code to make the motors move through a sequence of power values.  Try to avoid using *for()*, *while()*, or other blocking iterators within the _loop()_ function.  Instead, try using `if statements` and `global variables` to select different behaviours, and perhaps a variable to count iterations of `loop()`.

    - Get your Romi robot to drive in a straight line for a fixed period of time:
        - How reliably does it drive in a straight line?
            - Observe, does it always deviate in one direction?
        - Create a method to measure the deviation from a straight line path over a fixed distance.
        - Experiment to see how reliably you can compensate by adjusting the power values of your motors.
        - Does overall motor speed effect performance?
    - Using time or a count of `loop()` iterations, attempt to get your Romi to drive forwards, then backwards, and stop where it started:
        - How reliably can your Romi get back to where it started?
        - Determine the margin of error by conducting repeated tests.  Find a +/- mm estimate for the distance achieved away from its starting position. 
    - Can you get your Romi to rotate on the spot by 90 degrees?
        - How reliably can it rotate a fixed angle?
    - Can you get your Romi to drive the outline of a square?